# Implementation of baselines

#### See Section VI-B

In [1]:
import numpy as np
from utils import computeDegreeMatrix, spectralClustering
from sklearn.cluster import SpectralClustering
from tslearn.clustering import KernelKMeans
import kernel_kmeans

/Users/Meilame/opt/anaconda3/lib/python3.8/site-packages/tslearn/clustering/kmeans.py:20: UserWarning: Scikit-learn <0.24 will be deprecated in a future release of tslearn
  warnings.warn(


In [2]:
matrix = np.array([
    [[1,1,1,1,0,0,0,0],
     [1,1,1,1,0,0,0,0],
     [1,1,1,0,0,0,0,0],
     [1,1,1,1,1,0,0,0],
     [0,0,0,1,1,1,1,1],
     [0,0,0,0,1,1,1,1],
     [0,0,0,0,1,1,1,1],
     [0,0,0,0,1,1,1,1]],

    [[0,0,1,1,0,0,0,0],
     [1,1,1,1,0,0,0,0],
     [1,1,1,1,0,0,0,0],
     [1,1,1,1,1,0,0,0],
     [0,0,0,1,1,1,1,1],
     [0,0,0,0,1,1,1,1],
     [0,0,0,0,1,1,1,1],
     [0,0,0,0,1,1,1,1]],

    [[0,0,1,1,0,0,0,0],
     [1,1,1,1,0,0,0,0],
     [1,1,1,1,0,0,0,0],
     [1,1,1,1,1,0,0,0],
     [0,0,0,1,1,1,1,1],
     [0,0,0,0,1,1,1,1],
     [0,0,0,0,1,1,0,0],
     [0,0,0,0,1,1,1,1]]  ])

## SC-SUM

In [3]:
def SC_SUM(adj_matrix, k, normalized = False):
    """
    Spectral Clustering with summation of adjacency matrices
    See Eq 15 of the paper

    Parameters
    ----------
    adj_matrix : numpy array of shape (M,n,n)
    k (int): number of target clusters
    normalized (bool): whether to use normalized adjacency matrces

    Returns
    -------
    numpy array of shape (n,k) : cluster assignment matrix
    """
    M,n,_ = adj_matrix.shape # M is the number of clusters, n is the number of nodes
    
    if normalized:
        W = np.zeros((n,n))
        for i in range(M):
            W_i = adj_matrix[i,:,:]
            D_i = computeDegreeMatrix(W_i)
            W += (np.sqrt(np.linalg.inv(D_i))) @ W_i @ (np.sqrt(np.linalg.inv(D_i)))

        return spectralClustering(W, k)


    else:
        W = np.sum(adj_matrix, axis=0) #summation of the M adjacency matrices
        return spectralClustering(W, k)  

In [4]:
SC_SUM(matrix, 2, normalized = False)

array([0, 0, 0, 0, 1, 1, 1, 1], dtype=int32)

## K-KMeans

In [5]:
def K_KMeans(adj_matrix, d, k):
    """
    Kernel KMeans applied on the summation of spectral kernels of the adjacency matrices
    See Eq 17 of the paper

    Parameters
    ----------
    adj_matrix : numpy array of shape (M,n,n)
    d (int): number of eigenvectors to use
    k (int): number of target clusters

    Returns
    -------
    numpy array of shape (n,k) : cluster assignment matrix
    """
    M, n, _ = adj_matrix.shape

    W = np.zeros((n,n))
    for i in range(M):
        W_i = adj_matrix[i,:,:]
        D_i = computeDegreeMatrix(W_i)
        Lsym_i = (np.sqrt(np.linalg.inv(D_i))) @ (D_i - W_i) @ (np.sqrt(np.linalg.inv(D_i)))
        eigenvalues, eigenvectors = np.linalg.eig(Lsym_i)
        sorted_eigvecs = eigenvectors[:,np.argsort(eigenvalues)[:d]] #shape (n,d)

        K_i = np.zeros((n,n))
        for j in range(d):
            K_i += sorted_eigvecs[:,j] @ sorted_eigvecs[:,j].T
        
        W += K_i
    
    return SpectralClustering(n_clusters=k).fit(W).labels_
    #return KernelKMeans(n_clusters=k, kernel="rbf").fit(W).labels_
    #return kernel_kmeans.KernelKMeans(n_clusters = k).fit(W).labels_
    



In [6]:
K_KMeans(matrix, 8, 2)

/Users/Meilame/opt/anaconda3/lib/python3.8/site-packages/sklearn/cluster/_spectral.py:484: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn("The spectral clustering API has changed. ``fit``"


array([0, 1, 0, 1, 0, 0, 0, 0], dtype=int32)

## SC-AL

In [7]:
def SC_AL(adj_matrix, k):
    """
    Spectral Clustering with average of Laplacian matrices
    See Eq 18 of the paper

    Parameters
    ----------
    adj_matrix : numpy array of shape (M,n,n)
    k (int): number of target clusters

    Returns
    -------
    numpy array of shape (n,k) : cluster assignment matrix
    """
    
    M,n,_ = adj_matrix.shape # M is the number of clusters, n is the number of nodes

    #Computation of the average on the M Laplacian matrices
    W = np.zeros((n,n))
    for i in range(M):
        W_i = adj_matrix[i,:,:]
        D_i = computeDegreeMatrix(W_i)
        Lrw_i = np.linalg.inv(D_i) @ (D_i - W_i)
        W += Lrw_i
    W /= M

    return spectralClustering(W, k)

In [8]:
SC_AL(matrix, 2)

array([0, 1, 1, 1, 1, 1, 0, 1], dtype=int32)